In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category.v3/fact/granularity=monthly/date=2020-04-30/data_stage=final/device_code=android-all/
 


 



In [0]:

import datetime
start = "2015-08-01"
end = "2015-09-01"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
# sar_list=list()
# for days in xrange(date_range.days):
#     dates.append(real_date1 + datetime.timedelta(days))
#     if (real_date1 + datetime.timedelta(days)).weekday() == 5:
#         temp=list()
#         while dates:
#             temp.append(dates.pop())
#         sar_list.append({real_date1 + datetime.timedelta(days):temp})

month_day=list()
for days in xrange(date_range.days):
    month_day.append(real_date1 + datetime.timedelta(days))


test_list= sorted(list(set([ d.strftime("%Y-%m-%d")[:7] for d in month_day ])))

print test_list

In [0]:

from pyspark.sql import functions as F

test_list=["2019-09"]

def check_diff(month):
    df_1 = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}-*/".format(month))
    df_3 = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=monthly/date={}-*/".format(month))

    app_id_daily = df_1.createOrReplaceTempView("daily")
    app_id_monthly= df_3.createOrReplaceTempView("monthly")
    
    print "compare app id: " , month
    # spark.sql("select distinct app_id from monthly except all select distinct app_id from  daily ").show(2)
    # spark.sql("select distinct app_id from daily except all select distinct app_id from  monthly ").show(2)
    spark.sql("select app_id, country_code, sum(free_app_download) as free_app_download , sum(paid_app_download) as paid_app_download , sum(revenue) as revenue, device_code from daily group by country_code,device_code,app_id ").createOrReplaceTempView("sum_daily")
    # spark.sql("select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from sum_daily except all select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from monthly").show(2)
    # spark.sql("select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from monthly except all select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from sum_daily").show(2)

    except_1 = spark.sql("select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from sum_daily except all select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from monthly").withColumn("date", F.lit(month) ).withColumn("type",F.lit("daily_monthly"))
    except_2 = spark.sql("select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from monthly except all select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from sum_daily").withColumn("date", F.lit(month)).withColumn("type",F.lit("monthly_daily"))

    except_1.show()
    except_2.show()
    # df_write_result = except_2.union(except_1)
    # if df_write_result.rdd.isEmpty():
    #     print 'pass'
    # else:
    #     print 'failed!!!!!!!' , month
    # from aadatapipelinecore.core.utils.retry import retry
    # def write_test_result(df_write_result):
    #     df_write_result.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_monthly_count/",
    #                                       mode="append",
    #                                       partitionBy=["date"])
    # retry(write_test_result,(df_write_result,),{},interval=10)

sc.parallelize(map(check_diff, test_list), 1)

In [0]:
%python
import datetime
start = "2011-03-01"
end = "2020-04-09"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
# sar_list=list()
# for days in xrange(date_range.days):
#     dates.append(real_date1 + datetime.timedelta(days))
#     if (real_date1 + datetime.timedelta(days)).weekday() == 5:
#         temp=list()
#         while dates:
#             temp.append(dates.pop())
#         sar_list.append({real_date1 + datetime.timedelta(days):temp})

month_day=list()
for days in xrange(date_range.days):
    month_day.append(real_date1 + datetime.timedelta(days))


test_list= sorted(list(set([ d.strftime("%Y-%m-%d")[:7] for d in month_day ])))



In [0]:

from pyspark.sql import functions as F
import datetime
start_week = "2019-09-08"
end_week = "2019-09-15"

# start_week = "2019-10-20"
# end_week = "2019-10-27"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start_week.split('-')])
real_date2 = datetime.date(*[int(x) for x in end_week.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(dates.pop())
        sar_list.append({real_date1 + datetime.timedelta(days):temp})

test_path=list()
for x in sar_list:
    for key,item in x.items():
        test_path.append((key.strftime("%Y-%m-%d"),[d.strftime("%Y-%m-%d") for d in item]))



def check_diff(test_data):
    df_1 = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={%s}/"%(",".join(test_data[1])))
    df_3 = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=weekly/date={}/".format(test_data[0]))

    app_id_daily = df_1.createOrReplaceTempView("daily")
    app_id_monthly= df_3.createOrReplaceTempView("weekly")
    
    print "compare app id: " , test_data[0]
    # spark.sql("select distinct app_id from monthly except all select distinct app_id from  daily ").show(2)
    # spark.sql("select distinct app_id from daily except all select distinct app_id from  monthly ").show(2)
    spark.sql("select app_id, country_code, sum(free_app_download) as free_app_download , sum(paid_app_download) as paid_app_download , sum(revenue) as revenue, device_code from daily group by country_code,device_code,app_id ").createOrReplaceTempView("sum_daily")
    # spark.sql("select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from sum_daily except all select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from monthly").show(2)
    # spark.sql("select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from monthly except all select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from sum_daily").show(2)

    except_1 = spark.sql("select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from sum_daily except all select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from weekly").withColumn("date", F.lit(test_data[0]) ).withColumn("type",F.lit("daily_weekly"))
    except_2 = spark.sql("select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from weekly except all select app_id,country_code,free_app_download,paid_app_download,revenue,device_code from sum_daily").withColumn("date", F.lit(test_data[0])).withColumn("type",F.lit("weekly_daily"))

    df_write_result = except_2.union(except_1)
    if df_write_result.rdd.isEmpty():
        print 'pass'
    else:
        print 'failed!!!!!!!' , test_data[0]
    # from aadatapipelinecore.core.utils.retry import retry
    # def write_test_result(df_write_result):
    #     df_write_result.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_weekly_count/",
    #                                       mode="append",
    #                                       partitionBy=["date"])
    # retry(write_test_result,(df_write_result,),{},interval=10)

sc.parallelize(map(check_diff, test_path), 1)

In [0]:

spark.sql("select * from sum_daily where app_id=373998688 and country_code='US'").show()
spark.sql("select * from weekly where app_id= and country_code='US'").show()

In [0]:

print spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_weekly_count").select("date").orderBy("date").distinct().show(20)

In [0]:

print spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_monthly_count").select("date").orderBy("date").distinct().show(20)

In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=monthly/date=2019-10-31/

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/qa/temp/monthly/date=2019-09-30/device_code=android-all/
 
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/qa/temp/weekly/date=2019-09-28/
 


In [0]:
%%sh
